# Running "ELAND"
This notebook runs what is currently the very bare bones of ELAND

In [ ]:
# import libraries & scripts
import os
import pandas as pd
import numpy as np
import bihidef
from netZooPy import sambar
from goatools import obo_parser
from goatools.go_enrichment import GOEnrichmentStudy
from pybiomart import Dataset
import eland
import matplotlib.pyplot as plt
import seaborn as sns

#set working directory
os.chdir("results")

In [2]:
# set some params
panda_file = "gtex_breast_panda.txt"
panda_edgelist = "gtex_breast_edgelist.txt"
prior_file = "motif_prior_names_2024_filtered.txt"
edgelist_fil = "gtex_breast_edgelist_filtered.csv"
#edgelist_fil = "brca_fil_edges.csv"
#edgelist_top = "gtex_subset_first_half.csv"
#edgelist_end = "gtex_subset_end.csv"
#edgelist_end2 = "gtex_subset_end2.csv"

In [ ]:
# filter network
filter_panda.process_edge_list(panda_file, panda_edgelist)
fil_edges = filter_panda.filter_edges(prior_file=prior_file ,panda_file=panda_edgelist)
fil_edges.to_csv(edgelist_fil, index=False, header=False)

In [ ]:
# run bihidef
# this will save a bunch of files in the working directory
bihidef.bihidef(edgelist_fil, maxres=5)

In [2]:
# get communities
sign = pd.read_csv("pvg.nodes", delimiter = "\t")

# Extract clusters
clusters = sign.iloc[:, 0].astype(str)
clusters = clusters.str[7:]  # removing the first 7 characters
clusters = np.array([list(map(int, c.split('-'))) for c in clusters])

In [ ]:
# select clusters with sizes between selected range
# set range
min_size = 10
max_size = 200

fil_clust = sign[(sign.iloc[:,1] >= min_size) & (sign.iloc[:,1] <= max_size)]

#write to gmt
eland.gmt_from_bihidef(fil_clust, "fil_comm.gmt")

In [ ]:
#run sambar with bihidef communities
sambar.sambar(mut_file="../data/BRCAmutMatrixFinal.csv",
                            esize_file="../data/esizef.csv",
                            genes_file="../data/genes.txt",
                            gmtfile="fil_comm.gmt")

In [5]:
# run sambar with pathways
sambar.sambar(mut_file="../data/BRCAmutMatrixFinal.csv",
                            esize_file="../data/esizef.csv",
                            genes_file="../data/genes.txt",
                            gmtfile="../data/c2.cp.v2024.1.Hs.symbols.gmt")

Sambar runtime:  1.884204626083374
Clustering runtime:  199.0204849243164


(                                    TCGA-3C-AAAU-01A-11D-A41F-09  \
 BIOCARTA_41BB_PATHWAY                                        0.0   
 BIOCARTA_ACE2_PATHWAY                                        0.0   
 BIOCARTA_ACH_PATHWAY                                         0.0   
 BIOCARTA_ACTINY_PATHWAY                                      0.0   
 BIOCARTA_AGPCR_PATHWAY                                       0.0   
 ...                                                          ...   
 WP_WNT_SIGNALING_AND_PLURIPOTENCY                            0.0   
 WP_WNT_SIGNALING_IN_KIDNEY_DISEASE                           0.0   
 WP_WNT_SIGNALING_WP363                                       0.0   
 WP_WNT_SIGNALING_WP428                                       0.0   
 WP_ZINC_HOMEOSTASIS                                          0.0   
 
                                     TCGA-3C-AALI-01A-11D-A41F-09  \
 BIOCARTA_41BB_PATHWAY                                   0.000000   
 BIOCARTA_ACE2_PATHWAY          

# Go enrichment for selected communities

In [ ]:
# Load the GTEx gene annotation file
anno = pd.read_csv("../data/GTEx_gene_names.txt", delimiter="\t")

# Column 2: Summary of the number of genes in the community
print(sign.iloc[:, 1].describe())  # Summarizing the gene count per community

# Extract genes from the third column
allgenes = []
for i in range(len(fil_clust)):
    genes_in_com = fil_clust.iloc[i, 2]
    genes_in_com = genes_in_com.split(" ")  # Splitting by space
    allgenes.extend(genes_in_com)

allgenes = list(map(lambda x: x[:15], allgenes))  # Restricting to 15 characters
print(len(allgenes))  # Total genes
print(len(set(allgenes)))  # Unique genes

# Subset the annotation data for the genes in allgenes
anno_sub = anno[anno.iloc[:, 1].isin(allgenes)]
background = set(anno.iloc[:, 0])  # Using all genes as the background

In [5]:
# get go annotation
# Connect to the Ensembl human dataset using BioMart
dataset = Dataset(name='hsapiens_gene_ensembl', 
                  host='http://www.ensembl.org')

# Query BioMart for GO terms
gene_go_df = dataset.query(attributes=['ensembl_gene_id', 'go_id', 'hgnc_symbol'])

# filter based on your list of genes 
gene_go_df = gene_go_df[gene_go_df['HGNC symbol'].isin(allgenes)]

# remove any NAs
gene_go_df_clean = gene_go_df.dropna(subset=['GO term accession'])

print(gene_go_df_clean.head())  # Show the gene-to-GO mappings

# Convert DataFrame to a dictionary {gene_id: set([go_id1, go_id2, ...])}
gene_to_go_dict = gene_go_df_clean.groupby('Gene stable ID')['GO term accession'].apply(set).to_dict()


      Gene stable ID GO term accession HGNC symbol
501  ENSG00000288499        GO:0005576      FAM20C
502  ENSG00000288499        GO:0016020      FAM20C
503  ENSG00000288499        GO:0005783      FAM20C
504  ENSG00000288499        GO:0005654      FAM20C
505  ENSG00000288499        GO:0046872      FAM20C


In [14]:
# run go enrichment
godag = obo_parser.GODag("../data/go-basic.obo") 

goea = GOEnrichmentStudy(
        background, 
        gene_to_go_dict,  # Provide gene-to-GO mappings
        godag, 
        propagate_counts=True,
        methods=["bonferroni","fdr_bh"]
    )
    
goea_results = goea.run_study(study=gene_go_df_clean['Gene stable ID'])

../data/go-basic.obo: fmt(1.2) rel(2024-09-08) 44,296 Terms

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a


44 GO IDs NOT FOUND IN ASSOCIATION: GO:0005747 GO:0052895 GO:0060775 GO:0035308 GO:0052796 GO:1900022 GO:0102258 GO:0003867 GO:1902309 GO:0035509 GO:1902310 GO:0052857 GO:0032515 GO:0103066 GO:0052794 GO:0007205 GO:0000275 GO:0045281 GO:0103067 GO:0005750 GO:0032516 GO:0019391 GO:1904925 GO:0003402 GO:0008272 GO:0052795 GO:0102259 GO:0006211 GO:0102499 GO:0035410 GO:0102867 GO:0102773 GO:0052894 GO:0000276 GO:0019551 GO:0043666 GO:0052740 GO:0102250 GO:0035307 GO:0033878 GO:0004024 GO:0005355 GO:0090179 GO:0005746


  0%      0 of 30,243 population items found in association

Runing  Ontology Analysis: current study set of 115591 IDs.
  0%      0 of      0 study items found in association
  0%      0 of 115,591 study items found in population(30243)


# Comparing hierarchical clustering on community scores vs pathway scores
SAMBAR already outputs some clustering, so here just plotting heatmaps with dendograms.

In [2]:
# read in clusters
comm_clust = pd.read_csv("clustergroups_comm.csv", delimiter=",")
path_clust = pd.read_csv("clustergroups_path.csv", delimiter=",")


In [ ]:
# plot
sns.clustermap(path_clust)